# Database Check
Validate raw + derived tables and provenance for the FINRA + Polygon pipeline.


In [ ]:
from darkpool_analysis.config import load_config
from darkpool_analysis.db import get_connection
import pandas as pd

config = load_config()
conn = get_connection(config.db_path)


In [ ]:
tables = conn.execute("SHOW TABLES").df()["name"].tolist()
tables


In [ ]:
from IPython.display import display

for table in tables:
    print(table)
    display(conn.execute(f"DESCRIBE {table}").df())
    display(conn.execute(f"SELECT COUNT(*) AS n FROM {table}").df())


In [ ]:
sample_symbol = "AMZN"
start_date = min(config.target_dates)
end_date = max(config.target_dates)
sample_symbol, start_date, end_date


## Raw Tables Samples


In [ ]:
conn.execute("""
SELECT * FROM finra_otc_weekly_raw
WHERE symbol = ?
ORDER BY week_start_date DESC
LIMIT 5
""", [sample_symbol]).df()


In [ ]:
conn.execute("""
SELECT * FROM finra_short_daily_raw
WHERE symbol = ? AND trade_date BETWEEN ? AND ?
ORDER BY trade_date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


In [ ]:
conn.execute("""
SELECT * FROM polygon_daily_agg_raw
WHERE symbol = ? AND trade_date BETWEEN ? AND ?
ORDER BY trade_date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


In [ ]:
conn.execute("""
SELECT * FROM polygon_equity_trades_raw
WHERE symbol = ? AND timestamp::DATE BETWEEN ? AND ?
ORDER BY timestamp DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


## Derived Tables Samples


In [ ]:
conn.execute("""
SELECT * FROM lit_direction_daily
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
LIMIT 5
""", [sample_symbol, start_date, end_date]).df()


In [ ]:
conn.execute("""
SELECT * FROM daily_metrics
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
LIMIT 10
""", [sample_symbol, start_date, end_date]).df()


In [ ]:
conn.execute("""
SELECT index_symbol, trade_date, coverage_count, expected_constituent_count, coverage_pct
FROM index_constituent_short_agg_daily
ORDER BY trade_date DESC
LIMIT 5
""").df()


## Coverage and Provenance Checks


In [ ]:
conn.execute("""
SELECT symbol, COUNT(*) AS rows
FROM daily_metrics
WHERE date BETWEEN ? AND ?
GROUP BY symbol
ORDER BY symbol
""", [start_date, end_date]).df()


In [ ]:
conn.execute("""
SELECT date, symbol, otc_off_exchange_volume, otc_week_used, data_quality
FROM daily_metrics
WHERE symbol = ? AND date BETWEEN ? AND ?
ORDER BY date DESC
""", [sample_symbol, start_date, end_date]).df()


## Sanity Checks


In [ ]:
conn.execute("""
SELECT symbol, date, lit_buy_volume, lit_sell_volume, log_buy_sell
FROM lit_direction_daily
WHERE (lit_buy_volume <= 0 OR lit_sell_volume <= 0)
  AND log_buy_sell IS NOT NULL
LIMIT 10
""").df()


In [ ]:
conn.execute("""
SELECT symbol, date, short_ratio, short_ratio_denominator_type
FROM daily_metrics
WHERE short_ratio_denominator_type IS NOT NULL
  AND (short_ratio < 0 OR short_ratio > 1)
LIMIT 10
""").df()


In [ ]:
conn.execute("""
SELECT symbol, date, COUNT(*) AS n
FROM daily_metrics
GROUP BY symbol, date
HAVING n > 1
""").df()


In [ ]:
conn.execute("""
SELECT symbol, date, COUNT(*) AS n
FROM lit_direction_daily
GROUP BY symbol, date
HAVING n > 1
""").df()


## Lit Data Diagnostics
Checking lit_direction_daily and daily_metrics for lit flow issues.

In [ ]:
# 1. Check date range in lit_direction_daily (overall)
print("=== Date range in lit_direction_daily ===")
display(conn.execute("""
    SELECT MIN(date) as min_date, MAX(date) as max_date, COUNT(*) as total_rows
    FROM lit_direction_daily
""").df())

In [ ]:
# 2. Check which symbols have lit data
print("=== Symbols with lit data (lit_direction_daily) ===")
display(conn.execute("""
    SELECT symbol, COUNT(*) as rows, MIN(date) as min_date, MAX(date) as max_date
    FROM lit_direction_daily
    GROUP BY symbol
    ORDER BY rows DESC
""").df())

In [ ]:
# 3. Check lit_direction_daily for sample symbols (AAPL, SPY, XLF)
print("=== Lit data for sample symbols ===")
for sym in ["AAPL", "SPY", "XLF"]:
    print(f"\n--- {sym} ---")
    display(conn.execute("""
        SELECT date, lit_buy_volume, lit_sell_volume, lit_buy_ratio, log_buy_sell
        FROM lit_direction_daily
        WHERE symbol = ?
        ORDER BY date DESC LIMIT 10
    """, [sym]).df())

In [ ]:
# 4. Check daily_metrics lit columns for sample symbols
print("=== Daily_metrics lit columns for sample symbols ===")
for sym in ["AAPL", "SPY", "XLF"]:
    print(f"\n--- {sym} ---")
    display(conn.execute("""
        SELECT date, lit_flow_imbalance, lit_buy_ratio, lit_buy_volume, lit_sell_volume
        FROM daily_metrics
        WHERE symbol = ?
        ORDER BY date DESC LIMIT 10
    """, [sym]).df())

In [ ]:
# 5. Compare config target_dates with DB date ranges
print("=== Current config target_dates ===")
print(f"Min: {min(config.target_dates)}, Max: {max(config.target_dates)}")
print(f"Count: {len(config.target_dates)}")

print("\n=== Overlap check: lit_direction_daily dates within target range ===")
display(conn.execute("""
    SELECT COUNT(*) as matching_rows
    FROM lit_direction_daily
    WHERE date >= ? AND date <= ?
""", [min(config.target_dates), max(config.target_dates)]).df())

In [ ]:
# 6. Check polygon_ingestion_state (caching)
print("=== Polygon ingestion state (cached trades) ===")
display(conn.execute("""
    SELECT symbol, data_source, COUNT(*) as cached_dates, 
           MIN(trade_date) as min_date, MAX(trade_date) as max_date
    FROM polygon_ingestion_state
    GROUP BY symbol, data_source
    ORDER BY symbol
""").df())

In [ ]:
conn.close()
